# Tensorboard projection

In [23]:
import os
from copy import deepcopy
import pathlib
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from dashboard_helper import get_learner, inspect_runs, predict, get_transform_from_learner
from cotrain_helper import *
from paracoord_helper import *

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dataset = 'fused_sharp'
seed = 0

In [12]:
learner.cfg.todict()

{'DATA': {'DATABASE': '/home/zeyusun/work/flare-prediction-smarp/datasets/M_Q_24hr',
  'DATASET': 'fused_sharp',
  'AUXDATA': '/home/zeyusun/work/flare-prediction-smarp/datasets/sharp2smarp.npy',
  'BALANCED': True,
  'SEED': 0,
  'FEATURES': ['MAGNETOGRAM', 'USFLUXL', 'MEANGBL', 'R_VALUE', 'AREA'],
  'NUM_FRAMES': 1,
  'HEIGHT': 128,
  'WIDTH': 128,
  'BATCH_SIZE': 64,
  'NUM_WORKERS': 16,
  'TRANSFORMS': [],
  'SHRINKAGE': 'log',
  'THRESH': 150,
  'IMAGE_MEAN': -0.02334314373614411,
  'IMAGE_STD': 93.78056679862786,
  'CLASS_WEIGHT': [0.5, 0.5]},
 'LEARNER': {'CLASS_WEIGHT': 'balanced',
  'LEARNING_RATE': 0.001,
  'CHECKPOINT': '/home/zeyusun/work/flare-prediction-smarp/lightning_logs/version_959/checkpoints/epoch=19-step=3239.ckpt',
  'PATIENCE': 20,
  'VIS': {'GRADCAM_LAYERS': ['convs.conv5'],
   'ACTIVATIONS': ['convs.conv1', 'convs.conv2'],
   'HISTOGRAM': ['convs.conv3', 'linears.linear1']},
  'MODEL': {'NAME': 'CNN_Li2020', 'SETTINGS': 'cnn'}},
 'TRAINER': {'distributed_backen

In [25]:
from arnet.dataset import ActiveRegionDataModule
learner = get_learner(dataset, seed, 'CNN')
# hotfix
learner.cfg.DATA.DATABASE = '/home/zeyusun/work/flare-prediction-smarp/' + str(learner.cfg.DATA.DATABASE)
learner.cfg.DATA.AUXDATA = '/home/zeyusun/work/flare-prediction-smarp/' + str(learner.cfg.DATA.AUXDATA)
dm_learner = ActiveRegionDataModule(learner.cfg)

# High resolution, no transformation, include parameters
cfg = deepcopy(learner.cfg)
cfg.DATA.TRANSFORMS = []
cfg.DATA.FEATURES = ['MAGNETOGRAM', 'USFLUXL', 'MEANGBL', 'R_VALUE', 'AREA']
cfg.TRAINER.default_root_dir = 'lightning_logs_notebook'
dm = ActiveRegionDataModule(cfg)

Select iloc 0 from 
                         start_time tags.mlflow.runName  \
82 2021-06-27 16:03:33.239000+00:00            LSTM_CNN   

               tags.mlflow.source.git.commit  
82  e13810ffddafdb818df7c4de68d9eafd67861e8c  
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 1, 4, 4]         --
|    └─Conv3d: 2-1                       [-1, 64, 1, 128, 128]     7,808
|    └─BatchNorm3d: 2-2                  [-1, 64, 1, 128, 128]     128
|    └─ReLU: 2-3                         [-1, 64, 1, 128, 128]     --
|    └─MaxPool3d: 2-4                    [-1, 64, 1, 64, 64]       --
|    └─Conv3d: 2-5                       [-1, 64, 1, 64, 64]       102,464
|    └─BatchNorm3d: 2-6                  [-1, 64, 1, 64, 64]       128
|    └─ReLU: 2-7                         [-1, 64, 1, 64, 64]       --
|    └─MaxPool3d: 2-8                    [-1, 64, 1, 32, 32]       --
|    └─Conv3d: 2-9                       [-1, 64, 1,

/home/zeyusun/work/flare-prediction-smarp/arnet/dataset.py:158: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  self._construct_datasets(balanced=cfg.DATA.BALANCED)
/home/zeyusun/work/flare-prediction-smarp/arnet/dataset.py:158: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  self._construct_datasets(balanced=cfg.DATA.BALANCED)


In [14]:
from torchsummary import summary
summary(learner.model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv3d: 2-1                       7,808
|    └─BatchNorm3d: 2-2                  128
|    └─ReLU: 2-3                         --
|    └─MaxPool3d: 2-4                    --
|    └─Conv3d: 2-5                       102,464
|    └─BatchNorm3d: 2-6                  128
|    └─ReLU: 2-7                         --
|    └─MaxPool3d: 2-8                    --
|    └─Conv3d: 2-9                       36,928
|    └─BatchNorm3d: 2-10                 128
|    └─ReLU: 2-11                        --
|    └─MaxPool3d: 2-12                   --
|    └─Conv3d: 2-13                      36,928
|    └─BatchNorm3d: 2-14                 128
|    └─ReLU: 2-15                        --
|    └─MaxPool3d: 2-16                   --
|    └─Conv3d: 2-17                      36,928
|    └─BatchNorm3d: 2-18                 128
|    └─ReLU: 2-19                        --
|    └─MaxPool3d: 2-20                   --
├─

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv3d: 2-1                       7,808
|    └─BatchNorm3d: 2-2                  128
|    └─ReLU: 2-3                         --
|    └─MaxPool3d: 2-4                    --
|    └─Conv3d: 2-5                       102,464
|    └─BatchNorm3d: 2-6                  128
|    └─ReLU: 2-7                         --
|    └─MaxPool3d: 2-8                    --
|    └─Conv3d: 2-9                       36,928
|    └─BatchNorm3d: 2-10                 128
|    └─ReLU: 2-11                        --
|    └─MaxPool3d: 2-12                   --
|    └─Conv3d: 2-13                      36,928
|    └─BatchNorm3d: 2-14                 128
|    └─ReLU: 2-15                        --
|    └─MaxPool3d: 2-16                   --
|    └─Conv3d: 2-17                      36,928
|    └─BatchNorm3d: 2-18                 128
|    └─ReLU: 2-19                        --
|    └─MaxPool3d: 2-20                   --
├─

In [26]:
dl_test = dm_learner.test_dataloader()
# for batch in dl_test:
#     x = learner.convs(batch[0])
#     x = torch.flatten(x, 1)
#     x = learner.linears.linear1

In [52]:
dl_test.dataset[0][3]

0

In [60]:
from arnet.utils.gradcam import GradCAM
gradcam = GradCAM(learner.model, ['convs.conv4', 'linears.linear2'], 0, 1)
images = []
output = []
meta = []
for batch in tqdm(dl_test):
    videos = batch[0]
    _ = gradcam(videos.to('cuda'))
    images.append(videos[:,0])
    output.append(gradcam.activations['linears.linear2'])
    meta.append(batch[3])

100%|██████████| 13/13 [00:09<00:00,  1.32it/s]


In [68]:
mat = torch.cat(output) #.detach().cpu().numpy()
metadata = np.concatenate(meta)
label_img = torch.cat(images)

In [83]:
p = dm_learner.transform.transforms[1].__dict__
label_img = p['mean'] + label_img * p['std']

In [84]:
label_img = torch.clip(label_img, min=-300, max=300)
label_img = (label_img + 300) / 600

In [41]:
df_test = dl_test.dataset.df_sample

In [44]:
from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

In [85]:
writer.add_embedding(mat,
                     metadata=metadata,
                     label_img=label_img)
writer.close()

In [91]:
%tensorboard --logdir {writer.log_dir}

Reusing TensorBoard on port 6006 (pid 8717), started 0:50:47 ago. (Use '!kill 8717' to kill it.)

* Sample selection
    * Select only one image from each AR
    * Select only "pure" ARs (there shouldn't be many positive pure AR)
        * Get some statistics
    * Select those ARs are correctly predicted in each phase.

In [90]:
metadata = df_test.values.tolist()
metadata_header = df_test.columns.to_list()
writer.add_embedding(mat,
                     metadata=metadata,
                     label_img=label_img,
                     metadata_header=metadata_header,
                     tag='with header')
writer.close()

In [97]:
writer.log_dir

'runs/Jul26_00-41-04_napoli19'

In [99]:
writer.__dict__

{'log_dir': 'runs/Jul26_00-41-04_napoli19',
 'purge_step': None,
 'max_queue': 10,
 'flush_secs': 120,
 'filename_suffix': '',
 'file_writer': None,
 'all_writers': None,
 'default_bins': [-9.920775621859783e+19,
  -9.018886928963438e+19,
  -8.198988117239489e+19,
  -7.453625561126807e+19,
  -6.776023237388005e+19,
  -6.160021124898186e+19,
  -5.600019204452896e+19,
  -5.090926549502632e+19,
  -4.628115045002392e+19,
  -4.2073773136385384e+19,
  -3.824888466944125e+19,
  -3.477171333585568e+19,
  -3.1610648487141528e+19,
  -2.873695317012866e+19,
  -2.6124502881935143e+19,
  -2.3749548074486493e+19,
  -2.1590498249533174e+19,
  -1.962772568139379e+19,
  -1.7843386983085265e+19,
  -1.6221260893713875e+19,
  -1.4746600812467157e+19,
  -1.3406000738606506e+19,
  -1.2187273398733187e+19,
  -1.1079339453393805e+19,
  -1.0072126775812549e+19,
  -9.156478887102316e+18,
  -8.324071715547559e+18,
  -7.567337923225053e+18,
  -6.879398112022775e+18,
  -6.253998283657068e+18,
  -5.685452985142788e

In [100]:
%tensorboard --logdir runs/Jul26_00-41-04_napoli19

Reusing TensorBoard on port 6006 (pid 8717), started 0:54:48 ago. (Use '!kill 8717' to kill it.)